# Swap Network

Functionality for moving data between registers (swapping).

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np

## `CSwap`
Swap two registers controlled on a control bit.

Implements a multi-target controlled swap unitary $CSWAP_n = |0><0| I + |1><1| SWAP_n$.

This decomposes into a qubitwise SWAP on the two target registers, and takes $14n$ T-gates.

#### Parameters
 - `bitsize`: The bitsize of each of the two registers being swapped. 

#### Registers
 - `ctrl`: the control bit
 - `x`: the first register
 - `y`: the second register


In [ ]:
from qualtran.bloqs.basic_gates import CSwap

### Example Instances

In [ ]:
# A symbolic version. The bitsize is the symbol 'n'.
from sympy import sympify

cswap_symb = CSwap(bitsize=sympify('n'))

In [ ]:
# A small version on four bits.
cswap_small = CSwap(bitsize=4)

In [ ]:
# A large version that swaps 64-bit registers.
cswap_large = CSwap(bitsize=64)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([cswap_symb, cswap_small, cswap_large],
           ['`cswap_symb`', '`cswap_small`', '`cswap_large`'])

### Call Graph

In [ ]:
cswap_symb_g, cswap_symb_sigma = cswap_symb.call_graph()
show_call_graph(cswap_symb_g)
show_counts_sigma(cswap_symb_sigma)

## `CSwapApprox`
Approximately implements a multi-target controlled swap unitary using only $4n$ T-gates.

Implements $\mathrm{CSWAP}_n = |0 \rangle\langle 0| I + |1 \rangle\langle 1| \mathrm{SWAP}_n$
such that the output state is correct up to a global phase factor of +1 / -1.

This is useful when the incorrect phase can be absorbed in a garbage state of an algorithm
and thus ignored. See the reference for more details.

#### Parameters
 - `bitsize`: The bitsize of the two registers being swapped. 

#### Registers
 - `ctrl`: the control bit
 - `x`: the first register
 - `y`: the second register 

#### References
[Trading T-gates for dirty qubits in state preparation and unitary synthesis](https://arxiv.org/abs/1812.00954). Low et. al. 2018. See Appendix B.2.c.


In [ ]:
from qualtran.bloqs.swap_network import CSwapApprox

### Example Instances

In [ ]:
# A symbolic version. The bitsize is the symbol 'n'.
from sympy import sympify

approx_cswap_symb = CSwapApprox(bitsize=sympify('n'))

In [ ]:
# A small version on four bits.
approx_cswap_small = CSwapApprox(bitsize=4)

In [ ]:
# A large version that swaps 64-bit registers.
approx_cswap_large = CSwapApprox(bitsize=64)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([approx_cswap_symb, approx_cswap_small, approx_cswap_large],
           ['`approx_cswap_symb`', '`approx_cswap_small`', '`approx_cswap_large`'])

### Call Graph

In [ ]:
approx_cswap_symb_g, approx_cswap_symb_sigma = approx_cswap_symb.call_graph()
show_call_graph(approx_cswap_symb_g)
show_counts_sigma(approx_cswap_symb_sigma)

## `SwapWithZero`
Swaps |Psi_0> with |Psi_x> if selection register stores index `x`.

Implements the unitary U |x> |Psi_0> |Psi_1> ... |Psi_{n-1}> --> |x> |Psi_x> |Rest of Psi>.
Note that the state of `|Rest of Psi>` is allowed to be anything and should not be depended
upon.

#### References
[Trading T-gates for dirty qubits in state preparation and unitary synthesis] (https://arxiv.org/abs/1812.00954). Low, Kliuchnikov, Schaeffer. 2018.


In [ ]:
from qualtran.bloqs.swap_network import SwapWithZero

### Example Instances

In [ ]:
swz = SwapWithZero(selection_bitsize=8, target_bitsize=32, n_target_registers=4)

In [ ]:
# A small version on four bits.
swz_small = SwapWithZero(selection_bitsize=3, target_bitsize=2, n_target_registers=2)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([swz, swz_small],
           ['`swz`', '`swz_small`'])

### Call Graph

In [ ]:
swz_g, swz_sigma = swz.call_graph()
show_call_graph(swz_g)
show_counts_sigma(swz_sigma)